In [5]:
import re
import xml.dom.minidom

def ypath2xml(ypath, xmlns='', operation=None):
    #transforms xpath-like string (ypath) e.g. "/System/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni=10444"
    #to xml-like sequence of elements tags: 
    # <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    #   <eps-items>
    #       <epId-items>
    #           <Ep-list>
    #               <epId>1</epId>
    #               <nws-items>
    #                   <vni-items>
    #                       <Nw-list operation="remove">
    #                           <vni>10444</vni>
    #                       </Nw-list>
    #                   </vni-items>
    #               </nws-items>
    #           </Ep-list>
    #       </epId-items>
    #   </eps-items>
    # </System>
    #
    # it has optional parameter 'operation' that adds the string 'operation="value"' to the element marked with square brackets '[]'
    # for an nxos the value of the operation can be either "remove" or "replace"for an nxo
    #
    # there is a problem with parsing elements that contain the '/' separator within like this tDn element has:
    # /System/intf-items/svi-items/If-list/rtvrfMbr-items/tDn=/System/inst-items/Inst-list[name='{vrf_name}']
    # the same case if you want to configure a physical interface like Eth103/1/20
    # 
    # as a workaround I mark all the slashes with additional one, then replace that doubleslashes with # sign
    # then split ypath and unmark them back

    ypath = re.sub(r'//', '#', ypath) # <-- replace doubleslashes with '#'
    pl = ypath.split('/')
    
    xmls = f'<{pl[1]} xmlns="{xmlns}">' if xmlns else f'<{pl[1]}>'
    xmle = f'</{pl[1]}>'
     
    def _ypath2xml(pl):
        key = ''
        xmls = ''
        xmle = ''
        operation_set = ("remove", "replace")
        
        for i in range(len(pl)):
            elem = pl[i]
            if "=" in elem:
                elem,key = elem.split("=", 1)
                key = re.sub(r'#', '/', key) # <-- replace '#' with '/'
                xmls += f'<{elem}>{key}</{elem}>'
                break
            
            if "[]" in elem:
                elem = elem[:-2]
                if operation:
                    if operation not in operation_set:
                        raise ValueError(f'Incorrect operation value\nmust be one of the following: {", ".join(operation_set)}')
                    xmls += f'<{elem} operation="{operation}">'                
                else:
                    xmls += f'<{elem}>'
            else:                    
                xmls += f'<{elem}>'
            xmle = f'</{elem}>' + xmle
     
        if key and i < len(pl)-1:
            return xmls + _ypath2xml(pl[(i+1)::]) + xmle #recursion
        else:
            return xmls + xmle
         
    return xmls + _ypath2xml(pl[2::]) + xmle

def ppxml(xmlstr):
    print(xml.dom.minidom.parseString(xmlstr).toprettyxml(indent="    "))


In [6]:
from scrapli_netconf.driver import NetconfScrape
my_device = {
    "host": "192.168.99.91",
    "auth_username": "apiuser",
    "auth_password": "apipassword",
    "auth_strict_key": False,    
}

conn = NetconfScrape(**my_device)
conn.open()
xmlns = "http://cisco.com/ns/yang/cisco-nx-os-device"

In [7]:
# get all evpn related configs by netconf
vlan_id = 333
vxlan_id = 10333

xpath_nve = f"/System/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni={vxlan_id}"
xpath_svi = f"/System/intf-items/svi-items/If-list[]/id=vlan{vlan_id}"
xpath_vlan = f"/System/bd-items/bd-items/BD-list/fabEncap=vlan-{vlan_id}"
xpath_evpn = f"/System/evpn-items/bdevi-items/BDEvi-list[]/encap=vxlan-{vxlan_id}"

rpc_nve = ypath2xml(xpath_nve, xmlns)
rpc_svi = ypath2xml(xpath_svi, xmlns)
rpc_vlan = ypath2xml(xpath_vlan, xmlns)
rpc_evpn = ypath2xml(xpath_evpn, xmlns)

response = conn.get(filter_=rpc_nve, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_svi, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_vlan, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_evpn, filter_type="subtree")
print(response.result)

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="101">
  <data>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
      <eps-items>
        <epId-items>
          <Ep-list>
            <epId>1</epId>
            <nws-items>
              <vni-items>
                <Nw-list>
                  <vni>10333</vni>
                  <associateVrfFlag>false</associateVrfFlag>
                  <isLegacyMode>false</isLegacyMode>
                  <mcastGroup>230.1.1.3</mcastGroup>
                  <multisiteIngRepl>disable</multisiteIngRepl>
                  <spineAnyCastGw>false</spineAnyCastGw>
                  <suppressARP>off</suppressARP>
                </Nw-list>
              </vni-items>
            </nws-items>
          </Ep-list>
        </epId-items>
      </eps-items>
    </System>
  </data>
</rpc-reply>

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="102">
  <data>
    <System xmlns="http://cisco.com/ns/yang/cisco-n

In [8]:
# delete evpn
vlan_id = 333
vxlan_id = 10333

del_evpn_conf = ('<config><System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">' + 
        ypath2xml(f"/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni={vxlan_id}", operation="remove") + 
        ypath2xml(f"/intf-items/svi-items/If-list[]/id=vlan{vlan_id}", operation="remove") + 
        ypath2xml(f"/bd-items/bd-items/BD-list[]/fabEncap=vlan-{vlan_id}", operation="remove") + 
        ypath2xml(f"/evpn-items/bdevi-items/BDEvi-list[]/encap=vxlan-{vxlan_id}", operation="remove") + 
"</System></config>")


ppxml(del_evpn_conf)


<?xml version="1.0" ?>
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <eps-items>
            <epId-items>
                <Ep-list>
                    <epId>1</epId>
                    <nws-items>
                        <vni-items>
                            <Nw-list operation="remove">
                                <vni>10333</vni>
                            </Nw-list>
                        </vni-items>
                    </nws-items>
                </Ep-list>
            </epId-items>
        </eps-items>
        <intf-items>
            <svi-items>
                <If-list operation="remove">
                    <id>vlan333</id>
                </If-list>
            </svi-items>
        </intf-items>
        <bd-items>
            <bd-items>
                <BD-list operation="remove">
                    <fabEncap>vlan-333</fabEncap>
                </BD-list>
            </bd-items>
        </bd-items>
        <evpn-items>
          

In [9]:
response = conn.edit_config(config=del_evpn_conf, target="running")
print(response.result)

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="105">
  <ok/>
</rpc-reply>



In [10]:
conn.close()

In [12]:
xmlns = "http://cisco.com/ns/yang/cisco-nx-os-device"
xpath = "/System/intf-items/svi-items/If-list[]/id=vlan111/mtu=9000/descr=blablabla/adminSt=up/rtvrfMbr-items/tDn=//System//inst-items//Inst-list[name='Tenant-1']"

ppxml(ypath2xml(xpath, xmlns=xmlns, operation="replace"))

<?xml version="1.0" ?>
<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <intf-items>
        <svi-items>
            <If-list operation="replace">
                <id>vlan111</id>
                <mtu>9000</mtu>
                <descr>blablabla</descr>
                <adminSt>up</adminSt>
                <rtvrfMbr-items>
                    <tDn>/System/inst-items/Inst-list[name='Tenant-1']</tDn>
                </rtvrfMbr-items>
            </If-list>
        </svi-items>
    </intf-items>
</System>

